# Initialization

In [1]:
import tensorflow as tf

### Configuration
create_dataset = False

def in_google_colab():
    """Checks if the code is running in Google Colab

    Returns:
        bool: _description_
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

if in_google_colab():
    print("Running in Google Colab")
    # Install necessary packages in Google Colab
    !rm -r sample_data/
    !git clone https://github.com/naderinho/anesthesia_anomaly_detection
    !cp -r anesthesia_anomaly_detection/* .
    !rm -r anesthesia_anomaly_detection/
    !pip install vitaldb
    create_dataset = False
else:
    print("Running locally")

### Datasetpath
directory = 'data/'
datasetpath = 'dataset03/'
vitaldbpath = 'vitaldb_sevofl/'

### Import the necessary libraries
import numpy as np
import pandas as pd
import vitaldb as vf
import matplotlib.pyplot as plt
import pickle

### Custom functions
import modules as md

Running in Google Colab
Cloning into 'anesthesia_anomaly_detection'...
remote: Enumerating objects: 1180, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 1180 (delta 154), reused 198 (delta 139), pack-reused 963
Receiving objects: 100% (1180/1180), 241.60 MiB | 16.25 MiB/s, done.
Resolving deltas: 100% (685/685), done.
Updating files: 100% (556/556), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 12.6 MB/s eta 0:00:00


# Data loading

In [2]:
###### Create Dataset
if create_dataset:
    bis = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
    bis.name = 'Bispektralindex'
    bis.tracks = ['BIS/BIS']
    bis.filter = [20, 10, 100]
    bis.generateDataset(normalization=md.NormNone)
    bis.save('00_bis.npz')

    info = md.infoImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
    info.generateDataset(normalization=md.NormStandard)
    info.save('01_info.npz')

    bloodpressure = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
    bloodpressure.name = 'bloodpressure'
    bloodpressure.tracks = ['Solar8000/ART_DBP', 'Solar8000/ART_MBP', 'Solar8000/ART_SBP']
    bloodpressure.filter = [20, 20, 250]
    bloodpressure.generateDataset(normalization=md.NormStandard)
    bloodpressure.save('02_bloodpressure.npz')

    etCO2 = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
    etCO2.name = 'End Tidal CO2'
    etCO2.tracks = ['Primus/ETCO2']
    etCO2.filter = [5, 15, 50]
    etCO2.generateDataset(normalization=md.NormStandard)
    etCO2.save('02_etCO2.npz')

    spO2 = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
    spO2.name = 'SpO2'
    spO2.tracks = ['Solar8000/PLETH_SPO2']
    spO2.filter = [3, 80, 100]
    spO2.generateDataset(normalization=md.NormStandard)
    spO2.save('02_spO2.npz')

    hr = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
    hr.name = 'Heart Rate'
    hr.tracks = ['Solar8000/HR']
    hr.filter = [20, 40, 180]
    hr.generateDataset(normalization=md.NormStandard)
    hr.save('02_hr.npz')

    mac = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
    mac.name = 'MAC'
    mac.tracks = ['Primus/MAC']
    mac.filterON = False
    mac.generateDataset(normalization=md.NormNone)
    mac.save('03_mac.npz')

### Load the datasets
bis = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
bis.load('00_bis.npz')

info = md.infoImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
info.load('01_info.npz')

bloodpressure = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
bloodpressure.load('02_bloodpressure.npz')

etCO2 = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
etCO2.load('02_etCO2.npz')

spO2 = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
spO2.load('02_spO2.npz')

hr = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
hr.load('02_hr.npz')

mac = md.VitalImport(directory= directory, dataset=datasetpath, vitalpath=vitaldbpath)
mac.load('03_mac.npz')


train_index, val_index, test_index = bis.split(np.array(bis.index))

# Model creation

In [3]:
########################################## COMBINED MODEL ##########################################
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, LSTM, Dense, ReLU, Dropout, Concatenate, Masking, Conv1D, MaxPooling1D, BatchNormalization, RepeatVector, Lambda
from tensorflow.keras.metrics import RootMeanSquaredError, MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError

### Blood Pressure Input
input_bp = Input(shape=(None, bloodpressure.train_dataset.shape[2]))
bp_layer = Masking(mask_value=0.0)(input_bp)

bp_layer = LSTM(units=64, return_sequences=True)(bp_layer)
bp_layer = BatchNormalization()(bp_layer)
bp_layer = LSTM(units=64, return_sequences=True)(bp_layer)
bp_layer = BatchNormalization()(bp_layer)
bp_layer = LSTM(units=64, return_sequences=True)(bp_layer)
bp_layer = BatchNormalization()(bp_layer)
bp_layer = LSTM(units=64, return_sequences=True)(bp_layer)
bp_layer = BatchNormalization()(bp_layer)
bp_layer = LSTM(units=128, return_sequences=True)(bp_layer)
bp_layer = BatchNormalization()(bp_layer)
bp_layer = LSTM(units=64, return_sequences=True)(bp_layer)
bp_layer = BatchNormalization()(bp_layer)
bp_layer = LSTM(units=16, return_sequences=True)(bp_layer)
bp_layer = BatchNormalization()(bp_layer)
bp_layer = LSTM(units=16, return_sequences=True)(bp_layer)
bp_layer = BatchNormalization()(bp_layer)
bp_layer = Dense(units=16, activation='linear')(bp_layer)

### etCO2 Input
input_co2 = Input(shape=(None, etCO2.train_dataset.shape[2]))

co2_layer = LSTM(units=64, return_sequences=True)(input_co2)
co2_layer = BatchNormalization()(co2_layer)
co2_layer = LSTM(units=64, return_sequences=True)(co2_layer)
co2_layer = BatchNormalization()(co2_layer)
co2_layer = LSTM(units=64, return_sequences=True)(co2_layer)
co2_layer = BatchNormalization()(co2_layer)
co2_layer = LSTM(units=16, return_sequences=True)(co2_layer)
co2_layer = BatchNormalization()(co2_layer)
co2_layer = LSTM(units=8, return_sequences=True)(co2_layer)
co2_layer = BatchNormalization()(co2_layer)
co2_layer = Dense(units=8, activation='linear')(co2_layer)

### spo2 Input
input_spo2 = Input(shape=(None, spO2.train_dataset.shape[2]))

spo2_layer = LSTM(units=64, return_sequences=True)(input_spo2)
spo2_layer = BatchNormalization()(spo2_layer)
spo2_layer = LSTM(units=64, return_sequences=True)(spo2_layer)
spo2_layer = BatchNormalization()(spo2_layer)
spo2_layer = LSTM(units=64, return_sequences=True)(spo2_layer)
spo2_layer = BatchNormalization()(spo2_layer)
spo2_layer = LSTM(units=16, return_sequences=True)(spo2_layer)
spo2_layer = BatchNormalization()(spo2_layer)
spo2_layer = LSTM(units=8, return_sequences=True)(spo2_layer)
spo2_layer = BatchNormalization()(spo2_layer)
spo2_layer = Dense(units=8, activation='linear')(spo2_layer)

# Sevoflurane layers
input_mac = Input(shape=(None, mac.train_dataset.shape[2]))
mac_layer = BatchNormalization()(input_mac)
mac_layer = LSTM(units=64, return_sequences=True)(mac_layer)
mac_layer = BatchNormalization()(mac_layer)
mac_layer = LSTM(units=64, return_sequences=True)(mac_layer)
mac_layer = BatchNormalization()(mac_layer)
mac_layer = LSTM(units=64, return_sequences=True)(mac_layer)
mac_layer = BatchNormalization()(mac_layer)
mac_layer = LSTM(units=64, return_sequences=True)(mac_layer)
mac_layer = BatchNormalization()(mac_layer)
mac_layer = LSTM(units=128, return_sequences=True)(mac_layer)
mac_layer = BatchNormalization()(mac_layer)
mac_layer = LSTM(units=64, return_sequences=True)(mac_layer)
mac_layer = BatchNormalization()(mac_layer)
mac_layer = LSTM(units=16, return_sequences=True)(mac_layer)
mac_layer = BatchNormalization()(mac_layer)
mac_layer = LSTM(units=16, return_sequences=True)(mac_layer)
mac_layer = BatchNormalization()(mac_layer)
mac_layer = Dense(units=16, activation='linear')(mac_layer)

### INFO layers
input_info = Input(shape=(info.train_dataset.shape[1],))
info_layer = RepeatVector(bloodpressure.train_dataset.shape[1])(input_info)
info_layer = Dense(units=16, activation='linear')(info_layer)
info_layer = Dense(units=16, activation='sigmoid')(info_layer)


## Concatenate the Sevoflurane output with the info layer
comb_layer = Concatenate()([bp_layer, co2_layer, spo2_layer, mac_layer, info_layer])
comb_layer = Dense(units=64, activation='linear')(comb_layer)
comb_layer = Dense(units=64, activation='relu')(comb_layer)
comb_layer = Dense(units=64, activation='relu')(comb_layer)
comb_layer = Dense(units=32, activation='relu')(comb_layer)
comb_layer = Dense(units=16, activation='relu')(comb_layer)
comb_layer = Dense(units=1, activation=ReLU(max_value=1.0))(comb_layer)
output = Lambda(lambda x: x * 100)(comb_layer)

# Define the model
model = Model(inputs=[input_bp, input_co2, input_spo2, input_mac, input_info], outputs=output)

# Compile the model
optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['MeanSquaredError','MeanAbsoluteError','RootMeanSquaredError']
              )

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 3)]            0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, None, 1)]            0         []                            
                                                                                                  
 masking (Masking)           (None, None, 3)              0         ['input_1[0][0]']             
                                                                                                  
 batch_normalization_18 (Ba  (None, None, 1)              4         ['input_4[0][0]']             
 tchNormalization)                                                                            

# Model Training

In [5]:
# Rolling mean on BIS data
y = pd.DataFrame(bis.train_dataset[:,:,0].T).rolling(min_periods=1,window=3, center=True).mean().to_numpy().T[:,:,np.newaxis]

class SaveModelHistoryCallback(tf.keras.callbacks.Callback):
    def __init__(self, save_interval, model_path, history_path):
        super(SaveModelHistoryCallback, self).__init__()
        self.save_interval = save_interval
        self.model_path = model_path
        self.history_path = history_path
        self.history = []

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_interval == 0:
            # Save model
            self.model.save(self.model_path.format(epoch=epoch+1))
            # Save history
            self.history.append(logs)
            with open(self.history_path.format(epoch=epoch+1), 'wb') as f:
                pickle.dump(self.history, f)

save_interval = 20
model_path = 'model_epoch_{epoch}.h5'
history_path = 'history_epoch_{epoch}.pkl'
save_model_history_callback = SaveModelHistoryCallback(save_interval, model_path, history_path)



if in_google_colab():
    # Train the model
    history = model.fit([bloodpressure.train_dataset, etCO2.train_dataset, spO2.train_dataset, mac.train_dataset, info.train_dataset],
                        y,
                        validation_data=([bloodpressure.validation_dataset, etCO2.validation_dataset, spO2.validation_dataset, mac.validation_dataset, info.validation_dataset], bis.validation_dataset),
                        epochs=150,
                        callbacks=[save_model_history_callback],
                        batch_size=4
                        )

    train_score = history.history

    # Save the model
    model.save('download/model.keras')

    # Save the training history
    with open('download/train_score.pkl', 'wb') as f:
        pickle.dump(train_score, f)

    # Save the prediction
    y_pred = model.predict([bloodpressure.test_dataset, etCO2.test_dataset, spO2.test_dataset, mac.test_dataset, info.test_dataset], verbose=0)
    with open('download/prediction.pkl', 'wb') as f:
        pickle.dump(y_pred, f)

else:
    # Load train score data
    with open('train_score.pkl', 'rb') as f:
        train_score = pickle.load(f)

    # Load test prediction data
    with open('prediction.pkl', 'rb') as f:
        y_pred = pickle.load(f)

Epoch 1/150
47/47 [==============================] - 180s 2s/step - loss: 177.7199 - mean_squared_error: 177.7199 - mean_absolute_error: 6.3309 - root_mean_squared_error: 13.3312 - val_loss: 465.5007 - val_mean_squared_error: 465.5007 - val_mean_absolute_error: 19.4537 - val_root_mean_squared_error: 21.5755
Epoch 2/150
47/47 [==============================] - 97s 2s/step - loss: 55.5040 - mean_squared_error: 55.5040 - mean_absolute_error: 3.5222 - root_mean_squared_error: 7.4501 - val_loss: 465.8743 - val_mean_squared_error: 465.8743 - val_mean_absolute_error: 20.2397 - val_root_mean_squared_error: 21.5841
Epoch 3/150
47/47 [==============================] - 99s 2s/step - loss: 43.4577 - mean_squared_error: 43.4577 - mean_absolute_error: 3.1589 - root_mean_squared_error: 6.5922 - val_loss: 537.1931 - val_mean_squared_error: 537.1931 - val_mean_absolute_error: 21.1144 - val_root_mean_squared_error: 23.1774
Epoch 4/150
47/47 [==============================] - 100s 2s/step - loss: 39.7157

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


47/47 [==============================] - 98s 2s/step - loss: 27.7515 - mean_squared_error: 27.7515 - mean_absolute_error: 2.5960 - root_mean_squared_error: 5.2680 - val_loss: 75.6562 - val_mean_squared_error: 75.6562 - val_mean_absolute_error: 7.6793 - val_root_mean_squared_error: 8.6981
Epoch 21/150
47/47 [==============================] - 98s 2s/step - loss: 27.3559 - mean_squared_error: 27.3558 - mean_absolute_error: 2.5918 - root_mean_squared_error: 5.2303 - val_loss: 898.3641 - val_mean_squared_error: 898.3641 - val_mean_absolute_error: 25.3282 - val_root_mean_squared_error: 29.9727
Epoch 22/150
47/47 [==============================] - 99s 2s/step - loss: 28.0919 - mean_squared_error: 28.0919 - mean_absolute_error: 2.6153 - root_mean_squared_error: 5.3002 - val_loss: 29.6892 - val_mean_squared_error: 29.6892 - val_mean_absolute_error: 2.5215 - val_root_mean_squared_error: 5.4488
Epoch 23/150
47/47 [==============================] - 99s 2s/step - loss: 27.3290 - mean_squared_error:

# Training results

In [ ]:
from utils.plotting import training_loss_plot

plot = training_loss_plot(train_score, filename='download/training_loss.pdf')
plot.show()

# Testing

In [ ]:
### Predict on the test set
from utils.evaluation import phases_report, phases_report_std

print('Testmetriken:')

report = phases_report(y_pred, bis.test_dataset, mac.test_dataset)
report

In [ ]:
phases_report_std(report, y_pred, bis.test_dataset, mac.test_dataset)

In [ ]:
from utils.plotting import full_histogramm_plot

plot = full_histogramm_plot(groundtruth = bis.test_dataset, prediction = y_pred, filename='download/histogramm.pdf')
plot.show()

In [ ]:
from utils.plotting import single_prediction_plot

for case in test_index:
    single_prediction_plot(
        case = case,
        index = test_index,
        groundtruth = bis.test_dataset,
        prediction = y_pred,
        infusion = mac.test_dataset,
        error = 'Prediction RMSE',
        filename = 'download/' + str(case) + '.pdf')

print('Finished')

In [ ]:
from utils.plotting import full_prediction_plot

full_prediction_plot(index = test_index, groundtruth = bis.test_dataset, prediction = y_pred, infusion = mac.test_dataset)